# Shear profile around A360 using ComCam HSM shapes - Manual

Prakruth Adari, Céline Combet, Anja von der Linden\
LSST Science Piplines version: Weekly 2025_14\
Container Size: large

This notebook is a condensed set of code to obtain a shear profile of Abell 360. The main steps are:

- Loading the relevant object catalogs (all tracts and patches needed) using the butler
- Applying cuts (color/photo-z + lensing quality)
- Load in calibration (using the `gen_hsc_calibration` script)
- Shear profile

Most steps start with loading in data from the previous step so each step usually ends with writing data locally, this means we can quickly apply cuts to the same catalog and re-calibrate without having to re-query from the Butler. 

In [ ]:
!eups list -s | grep lsst_distrib

In [ ]:
import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from scipy.spatial import KDTree
import scipy.interpolate as interpolate
from scipy.optimize import curve_fit
import scipy.integrate as integrate
import scipy.stats as stats
import healpy as hp
import pandas as pd
from matplotlib import cm
from astropy.visualization import make_lupton_rgb
from matplotlib.legend_handler import HandlerLine2D
from astropy.table import Table, join, vstack
from astropy.io import fits
# %matplotlib widget

In [ ]:
# Familiar stack packages
from lsst.daf.butler import Butler
from lsst.geom import Box2I, Box2D, Point2I, Point2D, Extent2I, Extent2D
# from lsst.afw.image import Exposure, Image, PARENT
import lsst.sphgeom

# These may be less familiar objects dealing with multi-band data products
from lsst.afw.image import MultibandExposure, MultibandImage

In [ ]:
# import lsst.afw.display as afwDisplay
import os, sys

In [ ]:
hdir = os.getenv('HOME')
ddir = '/home/a/adari/DATA'

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
arcsec = 1/60**2

In [ ]:
rng = np.random.default_rng()

In [ ]:
def showRGB(image, bgr="gri", ax=None, fp=None, figsize=(8,8), stretch=57, Q=10):
    """Display an RGB color composite image with matplotlib.
    
    Parameters
    ----------
    image : `MultibandImage`
        `MultibandImage` to display.
    bgr : sequence
        A 3-element sequence of filter names (i.e. keys of the exps dict) indicating what band
        to use for each channel. If `image` only has three filters then this parameter is ignored
        and the filters in the image are used.
    ax : `matplotlib.axes.Axes`
        Axis in a `matplotlib.Figure` to display the image.
        If `axis` is `None` then a new figure is created.
    fp: `lsst.afw.detection.Footprint`
        Footprint that contains the peak catalog for peaks in the image.
        If `fp` is `None` then no peak positions are plotted.
    figsize: tuple
        Size of the `matplotlib.Figure` created.
        If `ax` is not `None` then this parameter is ignored.
    stretch: int
        The linear stretch of the image.
    Q: int
        The Asinh softening parameter.
    """
    # If the image only has 3 bands, reverse the order of the bands to produce the RGB image
    if len(image) == 3:
        bgr = image.filters
    # Extract the primary image component of each Exposure with the .image property, and use .array to get a NumPy array view.
    rgb = make_lupton_rgb(image_r=image[bgr[2]].array,  # numpy array for the r channel
                          image_g=image[bgr[1]].array,  # numpy array for the g channel
                          image_b=image[bgr[0]].array,  # numpy array for the b channel
                          stretch=stretch, Q=Q)  # parameters used to stretch and scale the pixel values
    if ax is None:
        fig = plt.figure(figsize=figsize)
        ax = fig.add_subplot(1,1,1)
    
    # Exposure.getBBox() returns a Box2I, a box with integer pixel coordinates that correspond to the centers of pixels.
    # Matplotlib's `extent` argument expects to receive the coordinates of the edges of pixels, which is what
    # this Box2D (a box with floating-point coordinates) represents.
    integerPixelBBox = image[bgr[0]].getBBox()
    bbox = Box2D(integerPixelBBox)
    ax.imshow(rgb, interpolation='nearest', origin='lower', extent=(bbox.getMinX(), bbox.getMaxX(), bbox.getMinY(), bbox.getMaxY()))
    if fp is not None:
        for peak in fp.getPeaks():
            ax.plot(peak.getIx(), peak.getIy(), "bx", mew=2)

In [ ]:
omega_m = .31
omega_de= .69
omega_r = 0
H0 = 70 # km/s/Mpc
c = 3e5 # km/s

In [ ]:
Hz = lambda z : c/(H0  * np.sqrt((omega_de + omega_m * (1+z)**3 + omega_r * (1+z)**4)))
chi_dl = lambda z, z0=0 : integrate.quad(Hz, z0, z)[0]
Da = lambda z : chi_dl(z)/(1+z)
beta_r = lambda zl, zs : integrate.quad(Hz, zl, zs)[0]/integrate.quad(Hz, 0, zs)[0]

In [ ]:
flux_suffix = '_cModelFlux'

In [ ]:
# Load operation rehearsal data
# Can use obs_butler.registry.queryCollections to see available collections

obs_repo = '/repo/main'
# obs_repo = '/LSSTComCam/DP1/defaults'
# obs_collection = 'LSSTComCamSim/runs/intermittentcumulativeDRP/20240402_03_04/d_2024_03_29/DM-43865'
# obs_collection = 'LSSTComCam/runs/nightlyValidation/20241108/d_2024_11_05/DM-47059'
# obs_collection = 'LSSTComCam/runs/DRP/20241101_20241211/w_2024_50/DM-48128'
# obs_collection = 'LSSTComCam/runs/DRP/20241101_20241127/w_2024_48/DM-47841'
# obs_collection = 'LSSTComCam/runs/DRP/DP1/w_2025_04/DM-48556'
# obs_collection = 'LSSTComCam/runs/DRP/DP1/w_2025_06/DM-48810'
# obs_collection = 'LSSTComCam/runs/DRP/DP1/w_2025_07/DM-48940'
# obs_collection = 'LSSTComCam/runs/DRP/DP1/w_2025_08/DM-49029'
# obs_collection = 'LSSTComCam/runs/DRP/DP1/w_2025_09/DM-49235' # DP1 version
# obs_collection = 'LSSTComCam/runs/DRP/DP1/w_2025_10/DM-49359'
# obs_collection = 'LSSTComCam/runs/DRP/DP1/w_2025_11/DM-49472'
# obs_collection = 'LSSTComCam/runs/DRP/DP1/w_2025_13/DM-49751'
# obs_collection = 'LSSTComCam/runs/DRP/DP1/v29_0_0_rc2/DM-49592'
obs_collection = 'LSSTComCam/runs/DRP/DP1/v29_0_0_rc5/DM-49865'

# obs_collection = 'LSSTComCam/runs/nightlyValidation/20241111/d_2024_11_05/DM-47059'

obs_butler = Butler(obs_repo, collections=obs_collection)
obs_registry = obs_butler.registry

In [ ]:
# cluster_coords = (59.48731586, -49.00034946)
cluster_coords = (37.86501659859067, 6.982204815599694)


## Load Butler Data

In [ ]:
t_skymap = obs_butler.get('skyMap', skymap='lsst_cells_v1')

In [ ]:
# Position of the BCG for A360
ra_bcg = cluster_coords[0]
dec_bcg = cluster_coords[1]

# Looking for all patches in delta deg region around it
delta = 0.5
center = lsst.geom.SpherePoint(ra_bcg, dec_bcg, lsst.geom.degrees)
ra_min, ra_max = ra_bcg - delta, ra_bcg + delta
dec_min, dec_max = dec_bcg - delta, dec_bcg + delta

ra_range = (ra_min, ra_max)
dec_range = (dec_min, dec_max)
radec = [lsst.geom.SpherePoint(ra_range[0], dec_range[0], lsst.geom.degrees),
         lsst.geom.SpherePoint(ra_range[0], dec_range[1], lsst.geom.degrees),
         lsst.geom.SpherePoint(ra_range[1], dec_range[0], lsst.geom.degrees),
         lsst.geom.SpherePoint(ra_range[1], dec_range[1], lsst.geom.degrees)]

tracts_and_patches = t_skymap.findTractPatchList(radec)

tp_dict = {}
for tract_num in np.arange(len(tracts_and_patches)):
    tract_info = tracts_and_patches[tract_num][0]
    tract_idx = tract_info.getId()
    # All the patches around the cluster
    patches = []
    for i,patch in enumerate(tracts_and_patches[tract_num][1]):
        patch_info = tracts_and_patches[tract_num][1][i]
        patch_idx = patch_info.sequential_index
        patches.append(patch_idx)
    tp_dict.update({tract_idx:patches})
print(tp_dict)

In [ ]:
tract = t_skymap.findTract(center)
patch = tract.findPatch(center)
tract_id = tract.tract_id
patch_id = patch.getSequentialIndex()
skymap_str = 'lsst_cells_v1'

In [ ]:
# Get the data for the first tract listed

deepCoaddTable = np.array(list(obs_registry.queryDatasets('deep_coadd', skymap='lsst_cells_v1', patch=patch_id, tract=tract_id)))

# deepCoaddTable = np.array(list(obs_registry.queryDatasets('deepCoadd_calexp', patch=patch_id, tract=tract_id)))
print(deepCoaddTable)

In [ ]:
new_deep_calexps = []
for dct in deepCoaddTable:
    new_deep_calexps.append(obs_butler.get(dct))

bands = [ndc.filter.bandLabel for ndc in new_deep_calexps]
coadds = MultibandExposure.fromExposures(bands, new_deep_calexps)
new_wcs = new_deep_calexps[bands.index('i')].getWcs()

In [ ]:
flux_bands = [b+suffix for b in bands for suffix in [flux_suffix, flux_suffix+'Err']]

In [ ]:
showRGB(coadds.image, bgr='gri', figsize=(6,6))

In [ ]:
req_cols = ['coord_ra', 'coord_dec', 'refExtendedness', 'merge_peak_sky', 'detect_isTractInner', 'detect_isDeblendedSource', 'objectId', 'detect_isPrimary',
            'parentObjectId', 'shape_xx', 'shape_xy', 'shape_yy', 'refBand', 'x', 'y', 'patch', 'i_ixxPSF', 'i_iyyPSF', 'i_ixyPSF', 'i_iPSF_flag',
            'i_hsmShapeRegauss_e1', 'i_hsmShapeRegauss_e2', 'i_hsmShapeRegauss_flag', 'i_hsmShapeRegauss_sigma', 'i_blendedness' ] + flux_bands

In [ ]:
cluster_tables = []

tids = tp_dict.keys()
# tids = [tract_id]
for tid in tids:
    for pi in tp_dict[tid]:
        cluster_tables.append(obs_butler.get('object_patch', skymap='lsst_cells_v1', patch=pi, tract=tid, 
                                   parameters={"columns":req_cols}))

In [ ]:
# cluster_table = pd.concat(cluster_tables)
cluster_table = vstack(cluster_tables, metadata_conflicts='silent')
primary_cluster = cluster_table[cluster_table['detect_isPrimary']]

In [ ]:
for b in bands:
    primary_cluster[f'{b}_mag'] = u.nJy.to(u.ABmag, primary_cluster[f'{b}{flux_suffix}'])
    primary_cluster[f'{b}_snr'] = primary_cluster[f'{b}{flux_suffix}']/primary_cluster[f'{b}{flux_suffix}Err']

In [ ]:
R2 = 1 - (primary_cluster['i_ixxPSF']+primary_cluster['i_iyyPSF'])/(primary_cluster['shape_xx']+primary_cluster['shape_yy'])
primary_cluster['res'] = R2

In [ ]:
primary_cluster.add_index('objectId')

In [ ]:
primary_cluster.meta = {'LSST.BUTLER.RUN': 'v29_0_0_rc5/DM-49865'}

In [ ]:
primary_cluster.write('cluster_data/abell360_PRECUTS_rc5.fits', format="fits", overwrite=True)

### Load Euclid Data

In [ ]:
with fits.open('cluster0.7_pz.fits') as hdul:
    # Assuming data is in the first HDU (if not, change the index as needed)
    data = hdul[1].data

    # Convert the FITS data to an Astropy Table
    euclid_pz = Table(data)


In [ ]:
euclid_coords = np.vstack((euclid_pz['right_ascension'], euclid_pz['declination'])).T
euclid_tree = KDTree(euclid_coords)

In [ ]:
primary_coords = np.vstack((primary_cluster['coord_ra'], primary_cluster['coord_dec'])).T
primary_tree = KDTree(primary_coords)

In [ ]:
euclid_match = euclid_tree.query(primary_coords, k=1)

In [ ]:
pz_est = []

for ed, em in zip(*euclid_match):
    if ed < arcsec:
        pz_est.append(euclid_pz[em]['phz_median'])
    else:
        pz_est.append(-99)
        

In [ ]:
primary_cluster['pz_euclid'] = pz_est

In [ ]:
plt.hist(primary_cluster['pz_euclid'], range=(0,4), bins=101);

In [ ]:
# # butler_run = primary_cluster.meta['LSST.BUTLER.RUN']
# primary_cluster.meta = {'LSST.BUTLER.RUN': 'v29_0_0_rc2/DM-49592'}

# primary_cluster['i_snr'] = primary_cluster['i_cModelFlux']/primary_cluster['i_cModelFluxErr']

# R2 = 1 - (primary_cluster['i_ixxPSF']+primary_cluster['i_iyyPSF'])/(primary_cluster['shape_xx']+primary_cluster['shape_yy'])
# primary_cluster['res'] = R2

In [ ]:
# Save matched catalog
primary_cluster.write('cluster_data/edfs0.7_cluster_euclidpz_PRECUTS.fits', format="fits", overwrite=True)

## Load-in PRECUTs

In [ ]:
# Load in pre-cuts data
with fits.open('cluster_data/abell360_PRECUTS_rc5.fits') as hdul:
        data = hdul[1].data
        table = Table(data)

# # Load in pre-cuts data
# with fits.open('cluster_data/edfs0.7_cluster_euclidpz_PRECUTS.fits') as hdul:
#         data = hdul[1].data
#         table = Table(data)

In [ ]:
color_ri = table['r_mag'] - table['i_mag']
color_gi = table['g_mag'] - table['i_mag']

In [ ]:
gi_rs = 1.6 - (0.07) * (table['i_mag']-21)
gi_rs_hi = gi_rs + 0.22
gi_rs_low = gi_rs - 0.12

gi_rs_lbl = ((color_gi > gi_rs_low) * (color_gi < gi_rs_hi) * (table['i_mag'] < 23))

ri_rs = 0.5 - (0.02) * (table['i_mag']-21)
ri_rs_hi = ri_rs + 0.05
ri_rs_low = ri_rs - 0.07

ri_rs_lbl = ((color_ri > ri_rs_low) * (color_ri < ri_rs_hi) * (table['i_mag'] < 23))

prak_rs = ri_rs_lbl * gi_rs_lbl

In [ ]:
color_ri = table['r_mag'] - table['i_mag']
c_rs_hi = 0.6 - (0.1/5.) * (table['r_mag']-19)
c_rs_low = 0.4 - (0.1/5.)* (table['r_mag']-19)

celine_rs = ((color_ri > c_rs_low) * (color_ri < c_rs_hi) * (table['r_mag'] < 23))

In [ ]:
# Apply cuts
filt = (table['refExtendedness'] >= 0.5)
filt &= (table['i_mag'] <= 24.5)
filt &= (table['i_snr'] >= 10)
filt &= ~(table['i_hsmShapeRegauss_flag'])
filt &= (table['i_hsmShapeRegauss_e1']**2 + table['i_hsmShapeRegauss_e2']**2) <= 16
filt &= table['res'] >= 0.3
filt &= table['i_blendedness'] <= 0.4
filt &= (table['i_hsmShapeRegauss_sigma']<= 0.4) * (0 < table['i_hsmShapeRegauss_sigma'])
filt &= table['i_iPSF_flag']==0
# filt &= ~celine_rs
filt &= ~prak_rs
# filt &= table['pz_euclid'] > .75
print(f"After cuts: {np.sum(filt)}")

In [ ]:
cut_table = table[filt]

In [ ]:
cut_table.write('cluster_data/abell360_POSTCUTS_colorcuts_all.fits', format="fits", overwrite=True)

## Load in calibration

In [ ]:
# From Shenming's CLMM demo on using HSC data
def apply_shear_calibration(e1_0, e2_0, e_rms, m, c1, c2, weight):
    R = 1.0 - np.sum(weight * e_rms**2.0) / np.sum(weight)
    m_mean = np.sum(weight * m) / np.sum(weight)
    c1_mean = np.sum(weight * c1) / np.sum(weight)
    c2_mean = np.sum(weight * c2) / np.sum(weight)
    print("R, m_mean, c1_mean, c2_mean: ", R, m_mean, c1_mean, c2_mean)

    g1 = (e1_0 / (2.0 * R) - c1) / (1.0 + m_mean)
    g2 = (e2_0 / (2.0 * R) - c2) / (1.0 + m_mean)

    return g1, g2

In [ ]:
# calib_filename = 'abell360_cc_all_calib.fits'
# postcuts_filename = 'abell360_POSTCUTS_colorcuts_all.fits'

calib_filename = 'source_sample_calib_v29_rc5.fits'
postcuts_filename = 'source_sample_v29_rc5.fits'

# calib_filename = 'edfs0.7_calibrate.fits'
# postcuts_filename = 'edfs0.7_cluster_euclidpz_POSTCUTS.fits'

In [ ]:
# with fits.open('cluster_data/edfs0.7_calibrate.fits') as hdul:
#     # Assuming data is in the first HDU (if not, change the index as needed)
#     data = hdul[1].data

#     # Convert the FITS data to an Astropy Table
#     table = Table(data)

# with fits.open('cluster_data/edfs0.7_cluster_euclidpz_POSTCUTS.fits') as hdul:
#     # Assuming data is in the first HDU (if not, change the index as needed)
#     data = hdul[1].data

#     # Convert the FITS data to an Astropy Table
#     wl_table = Table(data)

In [ ]:
with fits.open(f'cluster_data/{calib_filename}') as hdul:
    # Assuming data is in the first HDU (if not, change the index as needed)
    data = hdul[1].data

    # Convert the FITS data to an Astropy Table
    table = Table(data)

with fits.open(f'cluster_data/{postcuts_filename}') as hdul:
    # Assuming data is in the first HDU (if not, change the index as needed)
    data = hdul[1].data

    # Convert the FITS data to an Astropy Table
    wl_table = Table(data)

In [ ]:
e_rms = table["ishape_hsm_regauss_derived_rms_e"]
m = table["ishape_hsm_regauss_derived_shear_bias_m"]
c1 = table["ishape_hsm_regauss_derived_shear_bias_c1"]
c2 = table["ishape_hsm_regauss_derived_shear_bias_c2"]
weight = table["ishape_hsm_regauss_derived_shape_weight"]
# weight = np.ones(len(c1))

In [ ]:
g1, g2 = apply_shear_calibration(wl_table['i_hsmShapeRegauss_e1'], wl_table['i_hsmShapeRegauss_e2'], e_rms, m, c1, c2, weight)

# Profile

In [ ]:
def get_tang_cross(cluster, sky_dist, bins, sample_prefac=-1/2, astropy=True, ci_level=.95):
    nb = len(bins) - 1
    tang_avg = np.zeros(nb)
    cross_avg = np.zeros_like(tang_avg)

    tang_err = np.zeros((nb, 2))
    cross_err = np.zeros_like(tang_err)

    bin_rs = []
    for i in range(nb):
        bin_rs.append(bins[i])
        ndx_filt = (sky_dist > bins[i]) * (sky_dist < bins[i+1])
        if np.sum(ndx_filt) < 1:
            continue
        if astropy:
            sample = cluster[ndx_filt].data.data
        else:
            sample = cluster[ndx_filt]
        sample_t = sample_prefac*sample.real
        sample_x = sample_prefac*sample.imag

        # print(sample_t, sample_x)?

        ta = np.mean(sample_t)
        xa = np.mean(sample_x)

        te = stats.bootstrap([sample_t], np.mean, confidence_level=ci_level).confidence_interval
        xe = stats.bootstrap([sample_x], np.mean, confidence_level=ci_level).confidence_interval
        
        tang_avg[i] = ta
        cross_avg[i] = xa
        tang_err[i] = te
        cross_err[i] = xe

    return tang_avg, cross_avg, tang_err, cross_err, bin_rs

In [ ]:
# cluster_coords = (59.48731586, -49.00034946)
# cluster_coords = (37.86501659859067, 6.982204815599694)


In [ ]:
source_phi = np.arctan2(wl_table['coord_dec'] - cluster_coords[1], (cluster_coords[0] - wl_table['coord_ra'])*np.cos(np.deg2rad(cluster_coords[1])))
ang_dist = np.sqrt(((wl_table['coord_ra'] - cluster_coords[0]) * np.cos(np.deg2rad(cluster_coords[1])))**2 + (wl_table['coord_dec'] - cluster_coords[1])**2)
sky_distance = Da(.22) * ang_dist * (np.pi/180)

In [ ]:
trial_shear = g1+1j*g2
cl_shear = trial_shear * -1*np.exp(-2j*source_phi)

In [ ]:
bins_mpc = np.array([.25, .5, 1, 1.5, 2.27, 3.3, 5, 7])
bin_mids = 1/2 * (bins_mpc[1:] + bins_mpc[:-1])

In [ ]:
shear_cl = get_tang_cross(cl_shear, sky_distance, bins_mpc,sample_prefac=1, astropy=False, ci_level=.68)

In [ ]:
cmap = cm.coolwarm

# plt.plot(sky_distance, reduced_shear, '.')
plt.plot(bin_mids, shear_cl[0], '.', label='t', color=cmap(.05))
plt.vlines(bin_mids, shear_cl[2][:,0], shear_cl[2][:,1], color=cmap(0.05))

plt.plot(1.1*bin_mids, shear_cl[1], '.', label='x', color=cmap(.95))
plt.vlines(1.1*bin_mids, shear_cl[3][:,0], shear_cl[3][:,1], color=cmap(0.95))

# plt.plot(rbin_mids+.05*rbin_mids, crs_pshear[0], '.-', label='Céline - t', color=cmap(.15), alpha=.2)
# plt.vlines(rbin_mids+.05*rbin_mids, crs_pshear[2][:,0], crs_pshear[2][:,1], color=cmap(0.15), alpha=.2)

plt.semilogx()

plt.axhline(0, ls='--', color='k', alpha=.5)

# plt.errorbar(bin_mids*1.05, cl_plot['gt'], cl_plot['gt_err'], 
#              ls='', marker='.', label='Celine - t', color=cmap(0.15), alpha=.5)
# plt.errorbar(bin_mids*1.15, cl_plot['gx'], cl_plot['gx_err'], 
#              ls='', marker='.', label='Celine - x', color=cmap(0.95), alpha=.5)

# plt.plot(rbin_mids+.15*rbin_mids, crs_pshear[1], '.-', label='Céline - x', color=cmap(.95), alpha=.2)
# plt.vlines(rbin_mids+.15*rbin_mids, crs_pshear[3][:,0], crs_pshear[3][:,1], color=cmap(0.95), alpha=.2)
# plt.title("EDFS Cluster Profile at $z\\approx 0.7$")
plt.title("Abell 360")
# plt.xlim(0.2, 10)
plt.ylim([-0.03,0.08])
plt.xlim([0.3,7])
# plt.ylim(-0.05, 0.15)
plt.ylabel("Reduced shear")
plt.xlabel("R (Mpc)")
plt.legend(frameon=False)

#### CLMM Modeling

In [ ]:
import clmm
from clmm import GalaxyCluster, ClusterEnsemble, GCData, Cosmology
from clmm import Cosmology, utils

In [ ]:
cosmo = clmm.Cosmology(H0=70.0, Omega_dm0=0.3 - 0.045, Omega_b0=0.045, Omega_k0=0.0)

In [ ]:
moo = clmm.Modeling(massdef="mean", delta_mdef=200, halo_profile_model="nfw")

moo.set_cosmo(cosmo)
moo.set_concentration(4)
moo.set_mass(1.0e15)

z_cl = gc_object1.z

# source properties
# assume sources redshift following a the DESC SRD distribution. This will need updating.

z_distrib_func = utils.redshift_distributions.desc_srd  

# Compute first beta (e.g. eq(6) of WtGIII paper)
beta_kwargs = {
    "z_cl": z_cl,
    "z_inf": 10.0,
    "cosmo": cosmo,
    "z_distrib_func": z_distrib_func,
}
beta_s_mean = utils.compute_beta_s_mean_from_distribution(**beta_kwargs)
beta_s_square_mean = utils.compute_beta_s_square_mean_from_distribution(**beta_kwargs)

rproj = np.logspace(np.log10(0.3),np.log10(7.), 100)

gt_z = moo.eval_reduced_tangential_shear(
    rproj, z_cl, [beta_s_mean, beta_s_square_mean], z_src_info="beta", approx="order2"
)

In [ ]:
with fits.open('CLMM_profile.fits') as hdul:
    data = hdul[1].data
    cl_plot = Table(data)

In [ ]:
cmap = cm.coolwarm
cmap2 = cm.tab10

plt.plot(bin_mids, shear_cl[0], '.', label='Manual - t', color=cmap(.05))
plt.vlines(bin_mids, shear_cl[2][:,0], shear_cl[2][:,1], color=cmap(0.05))

plt.plot(1.1*bin_mids, shear_cl[1], '.', label='Manual - x', color=cmap(.95))
plt.vlines(1.1*bin_mids, shear_cl[3][:,0], shear_cl[3][:,1], color=cmap(0.95))

plt.errorbar(bin_mids*1.05, cl_plot['gt'], cl_plot['gt_err'], 
             ls='', marker='.', label='CLMM - t', color=cmap(0.15), alpha=.5)
plt.errorbar(bin_mids*1.15, cl_plot['gx'], cl_plot['gx_err'], 
             ls='', marker='.', label='CLMM - x', color=cmap(0.95), alpha=.5)

plt.plot(rproj, gt_z, label='NFW (model, not fit), M200m=1e15 Msun, c=4, n(z)=SRD', ls=':')


plt.semilogx()
plt.axhline(0, ls='--', color='k', alpha=.5)
plt.title("Abell 360 - Same Data")
# plt.xlim(0.2, 10)
plt.ylim([-0.03,0.08])
plt.xlim([0.3,7.5])
# plt.ylim(-0.05, 0.15)
plt.ylabel("Reduced shear")
plt.xlabel("R (Mpc)")
plt.legend(frameon=False, bbox_to_anchor=(1.05, .6))

### CLMM Profile

In [ ]:
import clmm
from clmm import GalaxyCluster, ClusterEnsemble, GCData, Cosmology
from clmm import Cosmology, utils

In [ ]:
cosmo = clmm.Cosmology(H0=70.0, Omega_dm0=0.3 - 0.045, Omega_b0=0.045, Omega_k0=0.0)

Prepare a CLMM GCData table using the catalog

In [ ]:
galcat = GCData()
galcat['ra'] = wl_table['coord_ra']
galcat['dec'] = wl_table['coord_dec']
# galcat['e1'] = e1[to_use]
# galcat['e2'] = e2[to_use]
galcat['e1'] = g1
galcat['e2'] = g2
# galcat['e_err'] = e_err[to_use]/2.  # factor 2 to account for conversion between e and g
galcat['z'] = np.zeros(len(g1)) # CLMM needs a redshift column for the source, even if not used

In [ ]:
ra_bcg = 37.86501659859067
dec_bcg = 6.982204815599694


In [ ]:
cluster_id = "Abell 360"
gc_object1 = clmm.GalaxyCluster(cluster_id, ra_bcg, dec_bcg, 0.22, galcat, coordinate_system='euclidean')

In [ ]:
gc_object1.compute_tangential_and_cross_components(add=True);

In [ ]:
gc_object1.make_radial_profile(bins=bins_mpc, bin_units='Mpc', add=True, cosmo=cosmo, overwrite=True, use_weights=False, gal_ids_in_bins=False);

## Alternatively, angular radial binning (no need for a cosmology then)
# gc_object1.make_radial_profile(bins=bins_deg, bin_units='degrees', add=True, overwrite=True, use_weights=True);


In [ ]:
# Check the profile table
gc_object1.profile

Also use CLMM to get a typical model for a cluster at that redshift, assuming the DESC SRD n(z)

In [ ]:
moo = clmm.Modeling(massdef="mean", delta_mdef=200, halo_profile_model="nfw")

moo.set_cosmo(cosmo)
moo.set_concentration(4)
moo.set_mass(1.0e15)

z_cl = gc_object1.z

# source properties
# assume sources redshift following a the DESC SRD distribution. This will need updating.

z_distrib_func = utils.redshift_distributions.desc_srd  

# Compute first beta (e.g. eq(6) of WtGIII paper)
beta_kwargs = {
    "z_cl": z_cl,
    "z_inf": 10.0,
    "cosmo": cosmo,
    "z_distrib_func": z_distrib_func,
}
beta_s_mean = utils.compute_beta_s_mean_from_distribution(**beta_kwargs)
beta_s_square_mean = utils.compute_beta_s_square_mean_from_distribution(**beta_kwargs)

rproj = np.logspace(np.log10(0.3),np.log10(7.), 100)

gt_z = moo.eval_reduced_tangential_shear(
    rproj, z_cl, [beta_s_mean, beta_s_square_mean], z_src_info="beta", approx="order2"
)

In [ ]:
plt.errorbar(gc_object1.profile['radius'], gc_object1.profile['gt'], gc_object1.profile['gt_err'], 
             ls='', marker='.', label='tangential')
plt.errorbar(gc_object1.profile['radius']*1.02, gc_object1.profile['gx'], gc_object1.profile['gx_err'], 
             ls='', marker='.', label='cross')
plt.plot(rproj, gt_z, label='NFW (model, not fit), M200m=1e15 Msun, c=4, n(z)=SRD', ls=':')

plt.xscale('log')
plt.axhline(0.0, color='k', ls=':')
plt.ylim([-0.03,0.08])
plt.xlim([0,7])
#plt.yscale('log')
plt.xlabel('R [Mpc]')
plt.ylabel('reduced shear')
plt.legend(loc=1)
plt.title("CLMM Profile")

# Playground

#### Red Sequence Snippet

In [ ]:
ang_dist = np.sqrt(((table['coord_ra'] - cluster_coords[0]) * np.cos(np.deg2rad(cluster_coords[1])))**2 + (table['coord_dec'] - cluster_coords[1])**2)
sub_table = table[(ang_dist < 10*60*arcsec)]

In [ ]:
# ang_dist = np.sqrt(((table['coord_ra'] - cluster_coords[0]) * np.cos(np.deg2rad(cluster_coords[1])))**2 + (table['coord_dec'] - cluster_coords[1])**2)
# sub_table = table[(ang_dist < 10*60*arcsec)]

xs = np.linspace(15, 28, num=1001)

gi_rs_plot = 1.6 - (0.07) * (xs-21)
gi_rs_hi_plot = gi_rs_plot + 0.22
gi_rs_low_plot = gi_rs_plot - 0.12

ri_rs_plot = 0.5 - (0.02) * (xs-21)
ri_rs_hi_plot = ri_rs_plot + 0.05
ri_rs_low_plot = ri_rs_plot - 0.07

zi_rs_plot = -.25 + (0.001) * (xs-21)
zi_rs_hi_plot = zi_rs_plot + 0.06
zi_rs_low_plot = zi_rs_plot - 0.08

fig, ax = plt.subplots(ncols=3, figsize=(12, 4))


ax[0].plot(sub_table['i_mag'], sub_table['r_mag'] - sub_table['i_mag'], '.', markersize=1)
ax[0].plot(xs, ri_rs_hi_plot, '--', color='grey')
ax[0].plot(xs, ri_rs_low_plot, '--', color='grey')

ax[1].plot(sub_table['i_mag'], sub_table['g_mag'] - sub_table['i_mag'], '.', markersize=1)
ax[1].plot(xs, gi_rs_hi_plot, '--', color='grey')
ax[1].plot(xs, gi_rs_low_plot, '--', color='grey')

ax[2].plot(sub_table['i_mag'], sub_table['z_mag'] - sub_table['i_mag'], '.', markersize=1)
ax[2].plot(xs, zi_rs_hi_plot, '--', color='grey')
ax[2].plot(xs, zi_rs_low_plot, '--', color='grey')

fig.suptitle("Abell 360 Red Sequence")
fig.supxlabel("i")
ax[0].set_ylabel("r-i")
ax[1].set_ylabel("g-i")
ax[2].set_ylabel("z-i")

for xx in ax:
    xx.axvline(24.5, ls='--', color='k', alpha=0.25)
    xx.set_xlim(15, 27)
    
ax[0].set_ylim(-1, 2)
ax[1].set_ylim(-1, 4)
ax[2].set_ylim(-1, 1)
plt.tight_layout()